In [1]:
import numpy as np
from PhysicsRegression import PhyReg

[2026-01-28 22:46:16,599] [    INFO] utils.py:148 - Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [2]:
phyreg = PhyReg(
    path = "./checkpoint.pth"
)

W0128 22:46:21.164673 118043 dynamic_loader.cc:412] The third-party dynamic library (libcuda.so) that Paddle depends on is not configured correctly. (error code is libcuda.so: cannot open shared object file: No such file or directory)
  Suggestions:
  1. Check if the third-party dynamic library (e.g. CUDA, CUDNN) is installed correctly and its version is matched with paddlepaddle you installed.
  2. Configure third-party dynamic library environment variables as follows:
  - Linux: set LD_LIBRARY_PATH by `export LD_LIBRARY_PATH=...`
  - Windows: set PATH by `set PATH=XXX;%PATH%`
  - Mac: set  DYLD_LIBRARY_PATH by `export DYLD_LIBRARY_PATH=...` [Note: After Mac OS 10.11, using the DYLD_LIBRARY_PATH is impossible unless System Integrity Protection (SIP) is disabled.]


AttributeError: 'dict' object has no attribute 'rescale'

## First try an toy example with Divide-and-Conquer Strategy

基本的符号回归流程，使用Oracle分治策略

数据准备了随机的x，用确定的函数目标函数 $y = 2.2 * x_0 * x_1$ 生成了 y，然后尝试拟合x和y的关系。


In [3]:
x = np.random.random((100, 2)) * 3
y = 2.2 * x[:, 0] * x[:, 1]

phyreg.fit(x, y, use_Divide=True, use_MCTS=False, use_GP=False,
           save_oracle_model=False, oracle_name="demo")

Training oracle Newral Network...
Generating formula through End-to-End...
Finished forward in 1.77109956741333 secs
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 0/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Back aggregating formulas...
idx : 0
expr: sqrt((4.840000000000001 * ((x_0)**2 * (x_1)**2)))
mse : 2.3977106336208324e-31



## Use the following code to try on your specified data.

- Modify the `oracle_name` parameter to point to another directory in `Oracle_model` for different symbolic regression problems

- Set `save_oracle_model` to be True if you wish to save to weight of OracleNN

In [ ]:
# your specific data

# x = ...
# y = ...

# assert x.shape[0] == y.shape[0] and len(y.shape) == 1

# phyreg.fit(x, y, use_Divide=True, use_MCTS=False, use_GP=False,
#            save_oracle_model=False, oracle_name="demo1")

## You can then utilize MCTS and GP algorism to further refine the results

还是一样的数据，但是加了蒙特卡洛树搜索和遗传编程优化，

In [4]:
x = np.random.random((100, 2)) * 3
y = 2.2 * x[:, 0] * x[:, 1]

phyreg.fit(x, y, use_Divide=True, use_MCTS=True, use_GP=True, use_pysr_init=True, use_const_optimization=True,
           save_oracle_model=False, oracle_name="demo")

Training oracle Newral Network...
Generating formula through End-to-End...
Finished forward in 1.6447875499725342 secs
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Back aggregating formulas...
idx : 0
expr: sqrt((4.840000000000001 * ((x_0)**2 * (x_1)**2)))
mse : 5.925109029432748e-31

Refining formula through MCTS...
idx : 0
expr: (2.2 * sqrt(((x_0)**2 * (x_1)**2)))
mse : 5.925109029432748e-31

Refining formula through GP...
idx : 0
expr: sqrt((4.840000000000001 * ((x_0)**2 * (x_1)**2)))
mse : 5.925109029432748e-31

Refining constants...
idx : 0
expr: sqrt((4.840000000000001 * ((x_0)**2 * (x_1)**2)))
mse : 5.925109029432748e-31



### Physical priors includes physical units, complexity, candidate unarys, candidate constants can also be included within the searching process.

In [5]:
phyreg.fit(
        x, y, use_Divide=True, use_MCTS=False, use_GP=False, 
        units = ["kg1m1s0T0V0", "kg0m0s2T-1V-1", "kg1m1s2T-1V-2"],
        complexitys = 8,
        unarys = ["sin", "cos"],
        consts = [[2.1, "kg0m0s0T0V-1"]],
        save_oracle_model=False, oracle_name="demo"
    )

Training oracle Newral Network...
Generating formula through End-to-End...
Finished forward in 3.9282021522521973 secs
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 0/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 0/2 skeleton duplicata
Removed 0/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Back aggregating formulas...
idx : 0
expr: (1.0248246858679373 * (x_0 * (x_1 * 2.146708632473897)))
mse : 5.113240804813337e-20



如果把上面的例子也用上所有的优化呢

In [ ]:
x = np.random.random((100, 2)) * 3
y = 2.2 * x[:, 0] * x[:, 1]

phyreg.fit(
    x,
    y,
    use_Divide=True,
    use_MCTS=True,
    use_GP=True,
    use_pysr_init=True,
    use_const_optimization=True,
    units=["kg1m1s0T0V0", "kg0m0s2T-1V-1", "kg1m1s2T-1V-2"],
    complexitys=8,
    unarys=["sin", "cos"],
    consts=[[2.1, "kg0m0s0T0V-1"]],
    save_oracle_model=False,
    oracle_name="demo",
)

Training oracle Newral Network...
Generating formula through End-to-End...
Finished forward in 4.060847043991089 secs
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 0/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 0/2 skeleton duplicata
Removed 0/2 skeleton duplicata
Removed 0/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Removed 1/2 skeleton duplicata
Back aggregating formulas...
idx : 0
expr: sqrt((7.634127474766057 * ((x_0)**2 * ((x_1)**2 * 0.6339951770517589))))
mse : 3.3968296968751206e-30

Refining formula through MCTS...
idx : 0
expr: (2.200000000000001 * sqrt(((x_0)**2 * (x_1)**2)))
mse : 3.3968296968751206e-30

Refining formula through GP...
idx : 0
expr: sqrt((7.634127474766057 * ((x_0)**2 * ((x_1)**2 * 0.6339951770517589))))
mse : 3.3968296968751206e-30

Refining constants...
idx : 0
expr: sqrt((7.634127474766057 * ((x_0)**2 * ((x_1)**2 * 0.6339951770517589))))
mse : 3.3968296968751206e-30



: 

还行，尽管表达式没化简，但是都是对的。